# Antibiotikaresistente Bakterier

Målet med dette prosjektet er å simulere evolusjon av antibiotika resistente bakterier og gjenskape et eksperiment gjort ved *Harvard Medical School*. Forklaringen av eksperimentet er utgangspunktet for modellen som skal brukes i dette prosjektet, så det er nødvendig å se videoen under før du går videre:

<iframe width="560" height="315" src="https://www.youtube.com/embed/plVk4NVIUh8" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

Når du har løst prosjektet, vil du ha produsert simuleringer som gir resultater som dette:


```{figure} ./animations/antibiotika_resistente_bakterier_xceller_300_yceller_150_niter_1000_ensidig.gif
---
name: bacteria
---

Animasjon av evolusjon av bakterier.
```

## Prosjektbeskrivelse

Du skal utvikle en Python-kode som simulerer bakterier i en petriskål der det er økende konsentrasjonsgrad av antibiotika i ulike soner, slik som i eksperimentet ved Harvard Medical School. Numeriske simuleringer har i stor grad fått stor plass som en form for eksperiment der man tester ut modeller med simuleringer for å studere egenskapene til modellen og sjekke om de samsvarer med observasjoner i naturen. Relatert til evolusjonsteorien, er det spesielt to betingelser som må være tilstede for at genforrådet til en art skal endre seg slik at vi observerer evolusjon:

1. Genetisk variasjon. Vi trenger et genforråd som har eller kan skape gener som gir forskjellige egenskaper hos arten. Mutasjon er en viktig komponent her, siden genforrådet kan endres ved at DNA til individer i en populasjon muterer og dermed endres.
2. Variasjon i miljø. Vi trenger et miljø som arten lever i der noen gener gir individer egenskaper som gir de en fordel med tanke på å overleve og reprodusere seg over andre individer i arten. 

Som mål skal du blant lage en animasjon og en figur som kartlegger populasjonsveksten over tid.

## Modell for bakterien

Her skal du sette opp modellen for bakterien og legge grunnlaget for simulering ved å skrive mindre funksjoner som løser deler av problemet. Til slutt skal du sy det sammen og simulere et system av mange bakterier. 

### En modell for bakterien og kode for modellen

Tenk deg at bakteriene levere i på en rektangulær plate. Tenk deg at vi deler opp langsiden i $L$ deler og kortsiden i $K$ deler. Tilsammen kan vi da tenke oss at det finnes $L \times K$ ruter som vi kan plassere en bakterie på. 

Når man skal sette lage en modell for et fysisk system, er første steg å forenkle og velge ut relevante variabler som vi tror har en effekt på hvordan systemet fungerer. I dette tilfellet er det fysiske systemet en bakterie. Hvis vi tenker oss om, bør vi i det minste passe på å ha en modell som fanger opp vanlige egenskaper hos en bakterie:

1. *Bevegelse*: Modellen for bakteriene må sørge for at bakteriene kan flytte seg rundt på en måte som replikerer bevegelsesmønstre til bakterier.
2. *Reproduksjon*: Modellen for bakteriene må ta hensyn til at bakteriene kan reprodusere gjennom ukjønnet formering.
3. *Mutasjon*: Bakteriene må kunne undergå genetiske mutasjoner slik at genetisk variasjon kan forekomme i populasjonen av bakterier.
4. *Dødsfall*: Bakteriene er nødt til å kunne dø i populasjonen.

### Bevegelse

Modellen for bakteriene må sørge for at forflytningen til bakteriene reflekterer hvordan **ekte** bakterier beveger seg. I praksis, vil bevegelsesmønsteret til bakterier opptre som svært tilfeldig. En enkel modell for bevegelsen til bakterien er derfor å generere hvilken retningen den går som **helt tilfeldig**. Det viser seg at dette garanterer at bakterien vil besøke alle mulige posisjoner på den rektangulære platen, et resultat som følger fra den matematisk teorien for **virrevandring**. Du kan derfor flytte bakteriene rundt helt tilfeldig. 

Siden bakterien lever på en rektangulær plate med $L$ posisjoner langs langsiden og $K$ posisjoner langs kortsiden, vil bakterien kunne leve på $L \times K$ ruter/celler i "Universet" til bakteriene. La oss kalle langsiden for $x$-retning og kortsiden for $y$-retning. Da kan bakterien ha posisjoner langs $x$-aksen med verdiene 

$$
x = 0, 1, 2, 3, \ldots, L - 1,
$$

og langs $y$-retningen 

$$
y = 0, 1, 2, 3, \ldots, K - 1.
$$

Vi kan derfor assosiere et koordinat $(x, y)$ som en beskrivelse av posisjonen til en bakterie på den rektangulære platen. 

Det er naturlig å benytte seg av et objekt i Python som heter *dictionary*, med datatype `dict`, som kan lagre denne informasjonen på følgende måte:

In [1]:
bakterie = {
    "x": 2,
    "y": 3,
}

I eksemplet her er `bakerie` plassert i koordinatet $(2, 3)$. Med andre ord er $x = 2$ og $y = 3$.
Hvis vi antar at bakterien **kun** kan flytte seg én plass av gangen i $x$- og $y$-retning, vil vi kunne flytte bakterien på følgende måte:

In [2]:
import numpy as np # bibliotek for numerisk matte
bakterie["x"] += np.random.randint(-1, 2) # Trekker tilfeldig en av tallene -1, 0 eller 1.
bakterie["y"] += np.random.randint(-1, 2)
print(bakterie)

{'x': 1, 'y': 3}


### Reproduksjon

En egenskap vi må ta hensyn til i en populasjon av bakterier er at når antall bakterier i populasjonen blir svært høy (og nærmer seg det maksimale antalle individer kalt **bæreevnen** til populasjonen), så vil sannsynligheten for at en bakterie i populasjon vil ha reprodusere bli svært lav. Dersom det er få individerer i populasjonen, derimot, vil sannsynligheten for reproduksjon av en tilfeldig bakterie være høy. Vi forventer at populasjonen følger en logistisk vekst, og vi kan sørge for at populasjonsveksten tilfredstiller en slik modell evd å benytte en sannsynlighetsfordeling for at en tilfeldig bakterie reproduserer ved

$$
p(n) = 2\left( 1 - \frac{1}{1 + e^{-b\cdot n}} \right), \qquad n \geq 0,
$$

der $n$ er antall individerer i bakteriepopulasjonen og $b$ er en parameter som bestemmer bæreevnene til populasjonen.

#### Hvordan sjekke om en bakterie kan reprodusere?

For å sjekke om en bakterie kan reprodusere, må du hente ut hvor mange bakterier som er i populasjonen. Vi skal se på hvordan vi gjør det senere. For nå, la oss bare anta at antall bakterier i populasjonen er $n$. For å sjekke om en bakterie for reprodusere, trekker vi da et tilfeldig tall $u$ mellom $0$ og $1$ (som vi kan gjøre med `u = np.random.uniform()`). Da gjelder følgende:

1. Hvis $p(n) > u$ så får bakterien reprodusert.
2. Hvis ikke er det for vanskelig for bakterien å reprodusere, og ingen reproduksjon vil skje.

### Hvordan lagre flere bakterier?

I lys av hva vi har sett på så langt, er det naturlig å lure på hvordan vi skal lagre mange bakterier av gangen. Tross alt består jo en populasjon av bakterier av flere enn én bakterie. Python bidrar med en nyttig datatype kalt for en *liste*, eller `list`, som gir oss evnen til å lagre flere ting i ett og samme objekt. 

For eksempel, kan vi vi lagre to bakterier på følgende vis:

In [4]:
bakterier = [
    {"x": 1, "y": 3}, # bakterie 1 på koordinat (1, 3)
    {"x": 3, "y": 5}, # bakterie 2 på koordinat (3, 5)
]